In [15]:
from pypdf import PdfReader
import re
import string
import os
from termcolor import colored

In [3]:
def is_newspaper_name(line):
    words = line.split()
    return len(words) < 5 and line[-1] not in string.punctuation

In [4]:
def replace_space_punct(chaine):
    # Créer un motif pour les espaces et les ponctuations
    motif = f"[{re.escape(string.punctuation)} ]+"
    # Remplacer les espaces et les ponctuations par un seul tiret
    resultat = re.sub(motif, "-", chaine)
    resultat = re.sub('’', "-", resultat)
    # Supprimer le tiret à la fin de la chaîne si présent
    return resultat.rstrip("-")

In [17]:
def article2txt(pdf_path, month:str, year:str) :
    reader = PdfReader(pdf_path)
    new_page = True
    folder = 'Sources/Europresse/txt/'+month+'_'+year
    os.makedirs(folder, exist_ok = True)
    for page in reader.pages:
        current_text = page.extract_text()
        if "Ce document référence un lien URL de site non hébergé par Cision." in current_text:
            print("Ce document référence un lien URL de site non hébergé par Cision.")
            continue
    
        text_lines = current_text.split("\n")
    
        if new_page and len(text_lines) > 3:
            content = ''
            newspaper = replace_space_punct(text_lines[0]) #on considère que le nom du journal est en première ligne et le titre de l'article en 4e ligne
            title = replace_space_punct(text_lines[3])
            first_line_num = 3
        else:
            first_line_num = 0
    
        while "-Aussi paru dans" in text_lines[-1] or is_newspaper_name(text_lines[-1]): #supprime les mentions de parution dans d'autres journaux
            text_lines.pop()
            try:
                text_lines[-1]
            except IndexError :
                print(colored("Erreur avec la lecture d'une ligne", 'red'))
                break
    
        for line_num in range(first_line_num, len(text_lines)):
            has_copyright = ('© ' in text_lines[line_num] and year in text_lines[line_num]) or ("Le présent document est protégé par les lois et conventions internationales sur le droit d'auteur et son utilisation est régie par ces lois et conventions" in text_lines[line_num])
            if not has_copyright and len(text_lines) > 3:
                content = content + ' ' + text_lines[line_num]
                if line_num == len(text_lines) - 1:
                    new_page = False
            else:
                #filename = PREF_POL_AUTEUR_ANNEE_MOIS_TITRE.[pdf/txt]
                pref = 'NEW_UNK_' + newspaper + '_' + year + '_' + month + '_'
                filename = folder + '/' + pref + title + '.txt'

                
                while True:
                    try:
                        i = 2
                        while(os.path.isfile(filename)):
                            filename = folder + '/' + pref + title + str(i) + '.txt'
                            i = i + 1
                        with open(filename, 'w+', encoding='utf-8') as f:
                            f.write(content)
                            print(f"Fichier {filename} enregistré")
                            break
                    except OSError as exc:
                        if exc.errno == 36:
                            title = title[0:int(len(title)/2)]
                            filename_tmp = 'NEW_UNK_' + newspaper + '_' + year + '_' + month + '_' + title + '.txt'
                            filename = folder + '/' + filename_tmp
                        else:
                            print("Erreur avec l'enregistrement du fichier")
                            break
                new_page = True
                break
            
            
        

In [18]:
for year in range(2019, 2025) :
    path = 'Sources/Europresse/pdf_print/' + str(year) + '/'
    for month in range(1, 13):
        month = str(month) if month > 9 else '0' + str(month)
        file = path + month + '_' + str(year) + '.pdf'
        if os.path.isfile(file):
            print(f"Ouverture fichier {file}")
            article2txt(file, month, str(year))
        i = 2
        while os.path.isfile(path + month + '_' + str(year) + '_' + str(i) + '.pdf'):
            print(f"Ouverture fichier {file}")
            article2txt(file, month, str(year))
            i = i + 1

Ouverture fichier Sources/Europresse/pdf_print/2019/01_2019.pdf
Fichier Sources/Europresse/txt/01_2019/NEW_UNK_Le-Journal-du-Centre_2019_01_L-acteur-né-à-Nevers-est-présélectionné-pour-le-César-du-meilleur-espoir-masculin.txt enregistré
Erreur avec la lecture d'une ligne
Fichier Sources/Europresse/txt/01_2019/NEW_UNK_Libération-site-web_2019_01_Le-dernier-exil-d-Edgar-HilsenrathL-écrivain-allemand-re.txt enregistré
Fichier Sources/Europresse/txt/01_2019/NEW_UNK_HuffPost-France-site-web_2019_01_Saverio-Tomasella.txt enregistré
Fichier Sources/Europresse/txt/01_2019/NEW_UNK_L-Histoire-no-455_2019_01_Les-Amérindiens-invisibles.txt enregistré
Fichier Sources/Europresse/txt/01_2019/NEW_UNK_L-Histoire-no-455_2019_01_Les-Amérindiens-invisibles2.txt enregistré
Fichier Sources/Europresse/txt/01_2019/NEW_UNK_Historia-no-865_2019_01_LES-RÉBUS-DE-ROMULUS-ET-REMUS.txt enregistré
Fichier Sources/Europresse/txt/01_2019/NEW_UNK_Mieux-Vivre-Votre-Argent-no-440_2019_01_Massonnaud-Robin.txt enregistré
Fi

KeyboardInterrupt: 

In [16]:
article2txt('Sources/Europresse/pdf_print/2019/01_2019.pdf', '01', '2019')

Fichier Sources/Europresse/txt/01_2019/NEW_UNK_Le-Journal-du-Centre_2019_01_L-acteur-né-à-Nevers-est-présélectionné-pour-le-César-du-meilleur-espoir-masculin.txt enregistré
Erreur avec la lecture d'une ligne
Fichier Sources/Europresse/txt/01_2019/NEW_UNK_Libération-site-web_2019_01_Le-dernier-exil-d-Edgar-HilsenrathL-écrivain-allemand-re.txt enregistré
Fichier Sources/Europresse/txt/01_2019/NEW_UNK_HuffPost-France-site-web_2019_01_Saverio-Tomasella.txt enregistré
Fichier Sources/Europresse/txt/01_2019/NEW_UNK_L-Histoire-no-455_2019_01_Les-Amérindiens-invisibles.txt enregistré
Fichier Sources/Europresse/txt/01_2019/NEW_UNK_L-Histoire-no-455_2019_01_Les-Amérindiens-invisibles2.txt enregistré
Fichier Sources/Europresse/txt/01_2019/NEW_UNK_Historia-no-865_2019_01_LES-RÉBUS-DE-ROMULUS-ET-REMUS.txt enregistré
Fichier Sources/Europresse/txt/01_2019/NEW_UNK_Mieux-Vivre-Votre-Argent-no-440_2019_01_Massonnaud-Robin.txt enregistré
Fichier Sources/Europresse/txt/01_2019/NEW_UNK_L-Histoire-no-455_2